In [18]:
use Dan;
use Dan::Pandas;

In [ ]:
say "=============================================";

### Series ###

In [19]:
my \s = $;
#my $index = {:a(0), :b(1), :c(2), :d(3), :e(4), :f(5)};
#s = Series.new(data => [1, 3, 5, NaN, 6, 8], :$index, name => 'john' );
#s = Series.new(data => [1, 3, 5, NaN, 6, 8], index => <a b c d e f>, name => 'john' );
#s = Series.new(data => [1, 3, 5, NaN, 6, 8]);
#s = Series.new([1, 3, 5, 6, 8]);
#s = Series.new([1, 3, 5, NaN, 6, 8]);
s = Series.new( [rand xx 5], index => <a b c d e>);
#s = Series.new( [b=>1, a=>0, c=>2] );               #from Array of Pairs
say ~s; 

a    0.880935
b    0.148055
c    0.986941
d    0.076753
e    0.622628
Name: anon, dtype: float64


In [20]:
say s.dtype;
say s.ix;
say s.index;
say ~s.reindex(['d','e','f','g','h','i']);
say s.elems;
say s.pull;
say ~s;

<class 'numpy.float64'>
[a b c d e]
{a => 0, b => 1, c => 2, d => 3, e => 4}
d    0.076753
e    0.622628
f         NaN
g         NaN
h         NaN
i         NaN
Name: anon, dtype: float64
5
[0.8809346416115266 0.14805482815455417 0.9869409564830176 0.07675327175046764 0.6226282455134127]
a    0.880935
b    0.148055
c    0.986941
d    0.076753
e    0.622628
Name: anon, dtype: float64


In [21]:
say s.map(*+2);
say [+] s; 
say s >>+>> 2; 
say s >>+<< s; 

say s[2];
say s<a>;

say ~s;

(2.8809346416115265 2.148054828154554 2.986940956483018 2.0767532717504675 2.622628245513413)
2.7153119435129787
(2.8809346416115265 2.148054828154554 2.986940956483018 2.0767532717504675 2.622628245513413)
(1.7618692832230531 0.29610965630910835 1.9738819129660352 0.1535065435009353 1.2452564910268253)
0.9869409564830176
0.8809346416115266
a    0.880935
b    0.148055
c    0.986941
d    0.076753
e    0.622628
Name: anon, dtype: float64


In [22]:
s.splice(1,2,(j=>3));

my \t = Series.new( [x=>1, y=>0, z=>2] );
s.concat: t;

say ~s;

a    0.880935
j    3.000000
d    0.076753
e    0.622628
x    1.000000
y    0.000000
z    2.000000
Name: anon, dtype: float64


In [23]:
say s.pd: '.shape';
say s.pd: '.flags';
say s.pd: '.T';
s.pd: '.to_json("test.json")';
s.pd: '.to_csv("test.csv")';
say s.pd: '.iloc[2] = 23';
say s.pd: '.iloc[2]';
say ~s;

a     0.880935
j     3.000000
d    23.000000
e     0.622628
x     1.000000
y     0.000000
z     2.000000
Name: anon, dtype: float64


In [27]:
# 2-arity pd methods
say ~my \quants = Series.new([100, 15, 50, 15, 25]);
say ~my \prices = Series.new([1.1, 4.3, 2.2, 7.41, 2.89]);

my \costs  = Series.new( quants >>*<< prices );

#-or- the pandas way
#my \costs = quants;
#costs.pd: '.mul', prices;

say "The costs are ...";
say ~costs;

0    100
1     15
2     50
3     15
4     25
Name: anon, dtype: int64
0    1.10
1    4.30
2    2.20
3    7.41
4    2.89
Name: anon, dtype: float64
The costs are ...
0    110.00
1     64.50
2    110.00
3    111.15
4     72.25
Name: anon, dtype: float64


In [28]:
my \u = s.Dan-Series;
say u.^name;
say ~u;

my \v = Series.new( u );
say v.^name;
say ~v;

Dan::Series
a	0.8809346416115266
j	3
d	23
e	0.6226282455134127
x	1
y	0
z	2
dtype: Num, name: anon

Dan::Pandas::Series
a     0.880935
j     3.000000
d    23.000000
e     0.622628
x     1.000000
y     0.000000
z     2.000000
Name: anon, dtype: float64


In [29]:
say "=============================================";

### DataFrames ###

In [30]:
my \dates = (Date.new("2022-01-01"), *+1 ... *)[^6];
my \df = DataFrame.new( [[rand xx 4] xx 6], index => dates, columns => <A B C D> );
#my \df = DataFrame.new( [[rand xx 4] xx 6], columns => <A B C D> );
#my \df = DataFrame.new( [[rand xx 4] xx 6] );
say ~df;

                   A         B         C         D
2022-01-01  0.450336  0.401494  0.431877  0.827657
2022-01-02  0.823830  0.768846  0.594041  0.658728
2022-01-03  0.190169  0.059628  0.382772  0.443158
2022-01-04  0.517103  0.291225  0.412561  0.892446
2022-01-05  0.952834  0.367124  0.889497  0.134775
2022-01-06  0.426687  0.383957  0.557565  0.969320


In [31]:
say "---------------------------------------------";

# Data Accessors [row;col]
say df[0;0];
df[0;0] = 3;                # set value (not sure why this works, must manual push

# Smart Accessors (mix Positional and Associative)
say df[0][0];
say df[0]<A>;
say df{"2022-01-03"}[1];

# Object Accessors & Slices (see note 1)
say ~df[0];                 # 1d Row 0 (DataSlice)
say ~df[*]<A>;              # 1d Col A (Series)
say ~df[0..*-2][1..*-1];    # 2d DataFrame
say ~df{dates[0..1]}^;      # the ^ postfix converts an Array of DataSlices into a new DataFrame

---------------------------------------------
0.45033620415325204
3
3
0.05962807473233611
A	3
B	0.40149355838665957
C	0.43187681631134633
D	0.8276566080719377
name: 2022-01-01

2022-01-01    0.450336
2022-01-02    0.823830
2022-01-03    0.190169
2022-01-04    0.517103
2022-01-05    0.952834
2022-01-06    0.426687
Name: A, dtype: float64
                   B         C         D
2022-01-01  0.401494  0.431877  0.827657
2022-01-02  0.768846  0.594041  0.658728
2022-01-03  0.059628  0.382772  0.443158
2022-01-04  0.291225  0.412561  0.892446
2022-01-05  0.367124  0.889497  0.134775
                   A         B         C         D
2022-01-01  0.450336  0.401494  0.431877  0.827657
2022-01-02  0.823830  0.768846  0.594041  0.658728


In [61]:
say "---------------------------------------------";
### DataFrame Operations ###

# 2d Map/Reduce
say df.map(*.map(*+2).eager);
say [+] df[*;1];
say [+] df[1;*];
say [+] df[*;*];

# Hyper
say df >>+>> 2;
say df >>+<< df;

# Describe
say ~df[0..^3]^;            # head
say ~df[(*-3..*-1)]^;       # tail
say ~df.shape;

---------------------------------------------
((2.450336204153252 2.823830030745463 2.1901689242092077 2.5171034417774836 2.952833987222898 2.4266867153436187) (2.4014935583866595 2.7688461834561178 2.059628074732336 2.291224991010741 2.367124178945053 2.383957080047151) (2.4318768163113464 2.5940411568261235 2.3827720931255536 2.4125614431968048 2.8894968296910655 2.557565128090037) (2.827656608071938 2.658728319165279 2.443158314541181 2.8924462004029943 2.1347748304656546 2.9693196952376204))
2.8454456901929834
2.272274066578059
12.827630805155579
([2.450336204153252 2.823830030745463 2.1901689242092077 2.5171034417774836 2.952833987222898 2.4266867153436187] [2.4014935583866595 2.7688461834561178 2.059628074732336 2.291224991010741 2.367124178945053 2.383957080047151] [2.4318768163113464 2.5940411568261235 2.3827720931255536 2.4125614431968048 2.8894968296910655 2.557565128090037] [2.827656608071938 2.658728319165279 2.443158314541181 2.8924462004029943 2.1347748304656546 2.9693196

In [63]:
df.describe;   #FIXME

In [64]:
# Transpose
say ~df.T;

                   A         B         C         D
2022-01-01  0.450336  0.401494  0.431877  0.827657
2022-01-02  0.823830  0.768846  0.594041  0.658728
2022-01-03  0.190169  0.059628  0.382772  0.443158
2022-01-04  0.517103  0.291225  0.412561  0.892446
2022-01-05  0.952834  0.367124  0.889497  0.134775
2022-01-06  0.426687  0.383957  0.557565  0.969320


In [65]:
say "---------------------------------------------";
# Sort

say ~df.sort: { .[1] };         # sort by 2nd col (ascending)
say ~df.sort: { -.[1] };        # sort by 2nd col (descending)
say ~df.sort: { df[$++]<C> };   # sort by col C
say ~df.sort: { df.ix[$++] };   # sort by index

# Grep (binary filter)
say ~df.grep( { .[1] < 0.5 } );                                # by 2nd column 
say ~df.grep( { df.ix[$++] eq <2022-01-02 2022-01-06>.any } ); # by index (multiple) 

---------------------------------------------
                   A         B         C         D
2022-01-03  0.190169  0.059628  0.382772  0.443158
2022-01-04  0.517103  0.291225  0.412561  0.892446
2022-01-05  0.952834  0.367124  0.889497  0.134775
2022-01-06  0.426687  0.383957  0.557565  0.969320
2022-01-01  0.450336  0.401494  0.431877  0.827657
2022-01-02  0.823830  0.768846  0.594041  0.658728
                   A         B         C         D
2022-01-02  0.823830  0.768846  0.594041  0.658728
2022-01-01  0.450336  0.401494  0.431877  0.827657
2022-01-06  0.426687  0.383957  0.557565  0.969320
2022-01-05  0.952834  0.367124  0.889497  0.134775
2022-01-04  0.517103  0.291225  0.412561  0.892446
2022-01-03  0.190169  0.059628  0.382772  0.443158
                   A         B         C         D
2022-01-03  0.190169  0.059628  0.382772  0.443158
2022-01-04  0.517103  0.291225  0.412561  0.892446
2022-01-01  0.450336  0.401494  0.431877  0.827657
2022-01-06  0.426687  0.383957  0.55

In [66]:
say "---------------------------------------------";

my \df2 = DataFrame.new([
        A => 1.0,
        B => Date.new("2022-01-01"),
        C => Series.new(1, index => [0..^4], dtype => Num),
        D => [3 xx 4],
        E => Categorical.new(<test train test train>),
        F => "foo",
]);
say ~df2;

---------------------------------------------
   A           B  C  D      E    F
0  1  2022-01-01  1  3   test  foo
1  1  2022-01-01  1  3  train  foo
2  1  2022-01-01  1  3   test  foo
3  1  2022-01-01  1  3  train  foo


No such symbol '<class 'numpy.int64'>'
  in block  at /usr/share/perl6/site/sources/A0EFFE63B40BC04CD8EB78CDCC7B22971E653B3A (Dan::Pandas) line 535
  in block  at /usr/share/perl6/site/sources/A0EFFE63B40BC04CD8EB78CDCC7B22971E653B3A (Dan::Pandas) line 533
  in block  at /usr/share/perl6/site/sources/A0EFFE63B40BC04CD8EB78CDCC7B22971E653B3A (Dan::Pandas) line 530
  in method TWEAK at /usr/share/perl6/site/sources/A0EFFE63B40BC04CD8EB78CDCC7B22971E653B3A (Dan::Pandas) line 516
  in method new at /usr/share/perl6/site/sources/A0EFFE63B40BC04CD8EB78CDCC7B22971E653B3A (Dan::Pandas) line 380
  in block <unit> at <unknown file> line 1
  in method eval at /usr/share/perl6/site/sources/FA76AD37A6E13534BC3F1D3E5089BFFB787F4AD0 (Jupyter::Kernel::Sandbox) line 122
  in code  at /usr/share/perl6/site/sources/42B1B7A6224CD62005B68B1EDFDE88F919C1BE32 (Jupyter::Kernel) line 146



In [67]:
say df2.data;
say df2.dtypes;
say df2.index;    #Hash (name => row number)   -or- df.ix; #Array
say df2.columns;  #Hash (label => col number)  -or- df.cx; #Array

[[1 2022-01-01 1 3 test foo] [1 2022-01-01 1 3 train foo] [1 2022-01-01 1 3 test foo] [1 2022-01-01 1 3 train foo]]
(Any)
{0 => 0, 1 => 1, 2 => 2, 3 => 3}
{A => 0, B => 1, C => 2, D => 3, E => 4, F => 5}


In [73]:
# row-wise splice:
#`[
my $ds = df2[0];                        # get a DataSlice 
$ds.splice($ds.index<A>,1,7);           # tweak it a bit
df2.splice( 1, 2, [j => $ds] );         # default
#]

# column-wise splice:
#`[
my $se = df2[*]<D>;                     # get a Series 
$se.splice(2,1,8);                      # tweak it a bit
df2.splice( :ax, 1, 2, [K => $se] );    # axis => 1
#]

say ~df2;   #FIXME

No such method 'key' for invocant of type 'Int'.  Did you mean 'keys'?

In [74]:
my \dfa = DataFrame.new(
        [['a', 1], ['b', 2]],
        columns => <letter number>,
);
say ~dfa;

my \dfc = DataFrame.new(
        [['c', 3, 'cat'], ['d', 4, 'dog']],
        columns => <animal letter number>,
);
say ~dfc;

say "---------------------------------------------";
dfa.concat(dfc);
say ~dfa;

  letter  number
0      a       1
1      b       2
  animal  letter number
0      c       3    cat
1      d       4    dog
---------------------------------------------
    letter number  animal
0        a      1     NaN
1        b      2     NaN
0⋅1    cat      c     3.0
1⋅1    dog      d     4.0


In [82]:
df.pd: '.shape';
df.pd: '.flags';
df.pd: '.T';
df.pd: '.to_json("test.json")';
df.pd: '.to_csv("test.csv")';
df.pd: '.iloc[2] = 23';
df.pd: '.iloc[2]';

say ~df;

                    A          B          C          D
2022-01-01   0.450336   0.401494   0.431877   0.827657
2022-01-02   0.823830   0.768846   0.594041   0.658728
2022-01-03  23.000000  23.000000  23.000000  23.000000
2022-01-04   0.517103   0.291225   0.412561   0.892446
2022-01-05   0.952834   0.367124   0.889497   0.134775
2022-01-06   0.426687   0.383957   0.557565   0.969320
